In [ ]:
Augment

In [18]:
import os
import cv2
import numpy as np
from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageEnhance
import random

# Configuration
dataset_path = "C:/Users/hp/music/face_recognition_project/face_recognition_in_jupyter/Datasset/train"
output_path = "C:/Users/hp/music/face_recognition_project/face_recognition_in_jupyter/Datasset/augmented"
augmentation_factor = 22  # Number of augmented images per original image
image_size = (224, 224)  # Increased size for better quality
use_pil_augmentation = True  # Set to False to use OpenCV/Keras method
jpeg_quality = 95  # Quality for saved JPEGs (1-100)

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

def high_quality_resize(image):
    """Resize image with high-quality interpolation"""
    return cv2.resize(image, image_size, interpolation=cv2.INTER_LANCZOS4)

def enhance_image_quality(image):
    """Apply quality enhancement to image"""
    # CLAHE for contrast enhancement
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    # Mild sharpening
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    image = cv2.filter2D(image, -1, kernel)
    
    return image

def keras_augmentation(imagePaths):
    """Augmentation using Keras ImageDataGenerator"""
    aug = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode="reflect",
        brightness_range=[0.9, 1.1],
        channel_shift_range=10.0,
        rescale=1./255,
        dtype='float32'
    )

    for (i, imagePath) in enumerate(imagePaths):
        label = imagePath.split(os.path.sep)[-2]
        output_dir = os.path.join(output_path, label)
        os.makedirs(output_dir, exist_ok=True)
        
        # Load and enhance original image
        image = cv2.imread(imagePath)
        image = enhance_image_quality(image)
        image = high_quality_resize(image)
        
        # Save original
        original_path = os.path.join(output_dir, f"original_{i}.jpg")
        cv2.imwrite(original_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg_quality])
        
        # Convert to array and expand dimensions for augmentation
        image_array = np.expand_dims(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), axis=0)
        
        # Generate augmented images
        total = 0
        for batch in aug.flow(image_array, 
                             batch_size=1,
                             save_to_dir=output_dir,
                             save_prefix=f"aug_{i}",
                             save_format="jpg"):
            total += 1
            if total >= augmentation_factor:
                break

        if (i + 1) % 5 == 0:
            print(f"[INFO] Processed {i + 1}/{len(imagePaths)} images")

def pil_augmentation(imagePaths):
    """Higher quality augmentation using PIL"""
    for (i, imagePath) in enumerate(imagePaths):
        label = imagePath.split(os.path.sep)[-2]
        output_dir = os.path.join(output_path, label)
        os.makedirs(output_dir, exist_ok=True)
        
        # Open and resize original image
        with Image.open(imagePath) as img:
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            # High-quality resize
            img = img.resize(image_size, Image.LANCZOS)
            
            # Save original
            original_path = os.path.join(output_dir, f"original_{i}.jpg")
            img.save(original_path, quality=jpeg_quality)
            
            # Create augmented versions
            for aug_num in range(augmentation_factor):
                augmented = img.copy()
                
                # Random flip
                if random.random() > 0.5:
                    augmented = augmented.transpose(Image.FLIP_LEFT_RIGHT)
                
                # Random rotation (-15 to +15 degrees)
                angle = random.uniform(-15, 15)
                augmented = augmented.rotate(angle, resample=Image.BICUBIC, expand=False)
                
                # Random brightness and contrast
                brightness = random.uniform(0.9, 1.1)
                contrast = random.uniform(0.9, 1.1)
                
                enhancer = ImageEnhance.Brightness(augmented)
                augmented = enhancer.enhance(brightness)
                
                enhancer = ImageEnhance.Contrast(augmented)
                augmented = enhancer.enhance(contrast)
                
                # Random color balance adjustment
                r, g, b = augmented.split()
                r = r.point(lambda x: x * random.uniform(0.9, 1.1))
                g = g.point(lambda x: x * random.uniform(0.9, 1.1))
                b = b.point(lambda x: x * random.uniform(0.9, 1.1))
                augmented = Image.merge('RGB', (r, g, b))
                
                # Save augmented image
                aug_path = os.path.join(output_dir, f"aug_{i}_{aug_num}.jpg")
                augmented.save(aug_path, quality=jpeg_quality)
        
        if (i + 1) % 5 == 0:
            print(f"[INFO] Processed {i + 1}/{len(imagePaths)} images")

def main():
    imagePaths = sorted(list(paths.list_images(dataset_path)))
    
    print(f"[INFO] Found {len(imagePaths)} images in dataset")
    print(f"[INFO] Using {'PIL' if use_pil_augmentation else 'Keras'} augmentation method")
    print(f"[INFO] Generating {augmentation_factor} augmented versions per image")
    
    if use_pil_augmentation:
        pil_augmentation(imagePaths)
    else:
        keras_augmentation(imagePaths)
    
    print("[INFO] Augmentation completed successfully!")

if __name__ == "__main__":
    main()

[INFO] Found 421 images in dataset
[INFO] Using PIL augmentation method
[INFO] Generating 22 augmented versions per image
[INFO] Processed 5/421 images
[INFO] Processed 10/421 images
[INFO] Processed 15/421 images
[INFO] Processed 20/421 images
[INFO] Processed 25/421 images
[INFO] Processed 30/421 images
[INFO] Processed 35/421 images
[INFO] Processed 40/421 images
[INFO] Processed 45/421 images
[INFO] Processed 50/421 images
[INFO] Processed 55/421 images
[INFO] Processed 60/421 images
[INFO] Processed 65/421 images
[INFO] Processed 70/421 images
[INFO] Processed 75/421 images
[INFO] Processed 80/421 images
[INFO] Processed 85/421 images
[INFO] Processed 90/421 images
[INFO] Processed 95/421 images
[INFO] Processed 100/421 images
[INFO] Processed 105/421 images
[INFO] Processed 110/421 images
[INFO] Processed 115/421 images
[INFO] Processed 120/421 images
[INFO] Processed 125/421 images
[INFO] Processed 130/421 images
[INFO] Processed 135/421 images
[INFO] Processed 140/421 images
[I

In [ ]:
training

In [3]:
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import datetime

# Configuration
CONFIG = {
    "dataset_path": "C:/Users/hp/music/face_recognition_project/face_recognition_in_jupyter/Datasset/Augmented",
    "output_dir": "C:/Users/hp/music/face_recognition_project/face_recognition_in_jupyter/",
    "image_size": (224, 224),
    "batch_size": 32,
    "epochs": 30,
    "test_size": 0.2,
    "random_state": 42
}

def create_output_dir(path):
    """Create output directory if it doesn't exist"""
    os.makedirs(path, exist_ok=True)
    print(f"[INFO] Output directory created at: {path}")

def load_dataset(dataset_path):
    """Load and preprocess dataset"""
    print(f"[INFO] Loading dataset from: {dataset_path}")
    imagePaths = list(paths.list_images(dataset_path))
    
    if not imagePaths:
        raise FileNotFoundError(f"No images found in {dataset_path}")
    
    data = []
    labels = []
    
    for imagePath in imagePaths:
        try:
            label = os.path.split(os.path.dirname(imagePath))[-1]
            image = cv2.imread(imagePath)
            
            if image is None:
                print(f"[WARNING] Could not read image: {imagePath}")
                continue
                
            image = cv2.resize(image, CONFIG["image_size"])
            data.append(image)
            labels.append(label)
            
        except Exception as e:
            print(f"[ERROR] Processing {imagePath}: {str(e)}")
            continue
    
    if not data:
        raise ValueError("No valid images found in dataset")
    
    return np.array(data), np.array(labels)

def build_model(num_classes):
    """Build the MobileNetV2 based model"""
    base_model = MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=(*CONFIG["image_size"], 3),
        pooling="avg"
    )
    base_model.trainable = False
    
    inputs = Input(shape=(*CONFIG["image_size"], 3))
    x = base_model(inputs, training=False)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs, outputs)
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

def save_artifacts(model, le, history):
    """Save model, labels, and training history"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(CONFIG["output_dir"], f"model_{timestamp}")
    create_output_dir(output_dir)
    
    try:
        # Save model
        model_path = os.path.join(output_dir, "face_recognition_model.h5")
        model.save(model_path)
        print(f"[SUCCESS] Model saved to: {model_path}")
        
        # Save label encoder classes
        classes_path = os.path.join(output_dir, "classes.npy")
        np.save(classes_path, le.classes_)
        print(f"[SUCCESS] Classes saved to: {classes_path}")
        
        # Save training history
        history_path = os.path.join(output_dir, "training_history.npy")
        np.save(history_path, history.history)
        print(f"[SUCCESS] Training history saved to: {history_path}")
        
        # Save configuration
        config_path = os.path.join(output_dir, "config.txt")
        with open(config_path, 'w') as f:
            for key, value in CONFIG.items():
                f.write(f"{key}: {value}\n")
        print(f"[SUCCESS] Config saved to: {config_path}")
        
    except Exception as e:
        print(f"[ERROR] Failed to save artifacts: {str(e)}")
        raise

def main():
    # Create output directory
    create_output_dir(CONFIG["output_dir"])
    
    # Load dataset
    X, y = load_dataset(CONFIG["dataset_path"])
    
    # Encode labels
    le = LabelEncoder()
    y = le.fit_transform(y)
    num_classes = len(le.classes_)
    y = to_categorical(y, num_classes=num_classes)
    
    print(f"[INFO] Found {num_classes} classes: {le.classes_}")
    
    # Split dataset
    (X_train, X_test, y_train, y_test) = train_test_split(
        X, y, 
        test_size=CONFIG["test_size"], 
        stratify=y, 
        random_state=CONFIG["random_state"]
    )
    
    # Normalize pixel values
    X_train = X_train.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0
    
    # Build model
    model = build_model(num_classes)
    model.summary()
    
    # Train model
    print("[INFO] Training model...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=CONFIG["batch_size"],
        epochs=CONFIG["epochs"],
        verbose=1
    )
    
    # Save artifacts
    save_artifacts(model, le, history)
    
    print("[INFO] Training completed successfully!")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] Training failed: {str(e)}")

[INFO] Output directory created at: C:/Users/hp/music/face_recognition_project/models/
[INFO] Loading dataset from: C:/Users/hp/music/face_recognition_project/face_recognition_in_jupyter/Datasset/Augmented
[INFO] Found 7 classes: ['Alemayehu' 'Banchiayehu' 'biniyam' 'martha' 'mikiyas' 'rahel' 'teferi']


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 1280)                │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7)                   │           8,967 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,266,951 (8.65 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

[INFO] Training model...


KeyboardInterrupt: 

In [ ]:
recognize image

In [7]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition System")
        self.root.geometry("630x600")  # Slightly taller for new elements
        
        # Configuration
        self.model_input_size = (224, 224)  # Must match your trained model
        self.display_size = (650, 360)      # Display resolution
        self.confidence_threshold = 0.7     # Default confidence level
        
        # Load model with error handling
        self.load_model_with_validation()
        
        # Initialize video
        self.cap = cv2.VideoCapture(0)
        self.video_running = False
        self.live_recognition = False
        self.current_frame = None
        self.photo = None
        
        # Create GUI
        self.create_widgets()
        
    def load_model_with_validation(self):
        """Safe model loading with validation"""
        try:
            self.model = load_model("face_recognition_model.h5")
            self.classes = np.load("classes.npy")
            print(f"Model loaded successfully. Recognizes {len(self.classes)} classes")
        except Exception as e:
            error_msg = f"Failed to load model: {str(e)}"
            print(error_msg)
            tk.messagebox.showerror("Error", error_msg)
            self.root.destroy()
    
    def create_widgets(self):
        """Build all GUI components"""
        # Main container
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Video display
        self.video_frame = tk.LabelFrame(self.main_frame, text="Camera Feed")
        self.video_frame.pack(pady=5)
        
        self.canvas = tk.Canvas(self.video_frame, width=self.display_size[0], 
                               height=self.display_size[1])
        self.canvas.pack()
        
        # Control panel
        self.control_frame = tk.LabelFrame(self.main_frame, text="Controls")
        self.control_frame.pack(fill=tk.X, pady=5)
        
        # Buttons
        buttons = [
            ("Start Camera", self.start_video),
            ("Stop Camera", self.stop_video),
            ("Live Recognition", self.toggle_live_recognition),
            ("Capture & Recognize", self.capture_face),
            ("Upload Image", self.upload_image)
        ]
        
        for i, (text, command) in enumerate(buttons):
            tk.Button(
                self.control_frame, text=text, width=15, 
                command=command
            ).grid(row=0, column=i, padx=5, pady=5)
        
        # Confidence threshold slider
        self.threshold_slider = tk.Scale(
            self.control_frame, 
            from_=0.5, to=0.95, 
            resolution=0.05,
            orient=tk.HORIZONTAL,
            label="Confidence Threshold",
            command=self.update_threshold
        )
        self.threshold_slider.set(self.confidence_threshold)
        self.threshold_slider.grid(row=1, column=0, columnspan=5, sticky="ew", padx=5)
        
        # Results display
        self.result_frame = tk.LabelFrame(self.main_frame, text="Recognition Result")
        self.result_frame.pack(fill=tk.X, pady=5)
        
        self.result_label = tk.Label(
            self.result_frame, text="No result yet", font=("Arial", 12))
        self.result_label.pack(pady=10)
        
        # Status bar
        self.status_bar = tk.Label(
            self.main_frame, text="Ready", 
            bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(fill=tk.X)
    
    def update_threshold(self, value):
        """Update confidence threshold from slider"""
        self.confidence_threshold = float(value)
        self.status_bar.config(text=f"Confidence threshold set to {value}")
    
    def start_video(self):
        """Start video capture"""
        if not self.video_running:
            self.video_running = True
            self.status_bar.config(text="Camera started")
            self.update_video()
    
    def stop_video(self):
        """Stop video capture"""
        self.video_running = False
        self.live_recognition = False
        self.status_bar.config(text="Camera stopped")
    
    def toggle_live_recognition(self):
        """Toggle real-time recognition"""
        self.live_recognition = not self.live_recognition
        if self.live_recognition:
            if not self.video_running:
                self.start_video()
            self.status_bar.config(text="Live recognition enabled")
        else:
            self.status_bar.config(text="Live recognition disabled")
    
    def update_video(self):
        """Update video frame"""
        if self.video_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_bar.config(text="Error reading camera")
                    return
                
                # Process frame
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, self.display_size)
                self.current_frame = frame.copy()
                
                # Apply recognition if enabled
                if self.live_recognition:
                    frame = self.process_frame(frame)
                
                # Display frame
                img = Image.fromarray(frame)
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)
                
                # Continue updating
                self.root.after(30, self.update_video)
            
            except Exception as e:
                self.status_bar.config(text=f"Camera error: {str(e)}")
                self.stop_video()
    
    def process_frame(self, frame):
        """Process frame for face recognition"""
        display_frame = frame.copy()
        
        try:
            # Prepare face for model
            face = cv2.resize(frame, self.model_input_size)
            face = face.astype("float32") / 255.0
            face = np.expand_dims(face, axis=0)
            
            # Predict
            preds = self.model.predict(face, verbose=0)[0]
            idx = np.argmax(preds)
            proba = preds[idx]
            name = self.classes[idx] if proba >= self.confidence_threshold else "Unknown"
            
            # Update UI
            result_text = (f"Recognized: {name} ({proba*100:.2f}%)" 
                          if name != "Unknown" else "Unknown face detected")
            self.result_label.config(text=result_text)
            
            # Annotate frame
            cv2.putText(
                display_frame, f"{name}: {proba*100:.2f}%",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                (0, 255, 0) if name != "Unknown" else (0, 0, 255), 2
            )
            
        except Exception as e:
            self.status_bar.config(text=f"Recognition error: {str(e)}")
        
        return display_frame
    
    def capture_face(self):
        """Capture and recognize current frame"""
        if self.current_frame is None:
            self.status_bar.config(text="No frame available to capture")
            return
        
        try:
            # Process frame
            face = cv2.resize(self.current_frame, self.model_input_size)
            face = face.astype("float32") / 255.0
            face = np.expand_dims(face, axis=0)
            
            # Predict
            preds = self.model.predict(face, verbose=0)[0]
            idx = np.argmax(preds)
            proba = preds[idx]
            name = self.classes[idx] if proba >= self.confidence_threshold else "Unknown"
            
            # Show result
            result_text = (f"Recognized: {name} ({proba*100:.2f}%)" 
                         if name != "Unknown" else "Unknown face detected")
            self.result_label.config(text=result_text)
            self.status_bar.config(text="Capture successful")
            
        except Exception as e:
            self.result_label.config(text="Recognition failed")
            self.status_bar.config(text=f"Error: {str(e)}")
    
    def upload_image(self):
        """Upload and recognize image file"""
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")])
        
        if not file_path:
            return
        
        try:
            # Read image
            image = cv2.imread(file_path)
            if image is None:
                raise ValueError("Could not read image file")
            
            # Convert and store
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.current_frame = image.copy()
            
            # Display with aspect ratio preservation
            h, w = image.shape[:2]
            aspect = w / h
            new_w = min(self.display_size[0], int(self.display_size[1] * aspect))
            new_h = min(self.display_size[1], int(self.display_size[0] / aspect))
            resized = cv2.resize(image, (new_w, new_h))
            
            # Center on canvas
            display_img = np.ones((self.display_size[1], self.display_size[0], 3), 
                                 dtype=np.uint8) * 255
            x_offset = (self.display_size[0] - new_w) // 2
            y_offset = (self.display_size[1] - new_h) // 2
            display_img[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
            
            # Show image
            img = Image.fromarray(display_img)
            self.photo = ImageTk.PhotoImage(image=img)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)
            
            # Recognize
            self.capture_face()
            self.status_bar.config(text=f"Loaded: {os.path.basename(file_path)}")
            
        except Exception as e:
            self.result_label.config(text="Image load failed")
            self.status_bar.config(text=f"Error: {str(e)}")
    
    def on_closing(self):
        """Cleanup on window close"""
        self.stop_video()
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        if hasattr(self, 'photo'):
            del self.photo
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = FaceRecognitionApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

Model loaded successfully. Recognizes 7 classes
